# 1. Cách bạn đang retrieval như nào ? @src 



Hệ thống sử dụng phương pháp kết hợp (Ensemble) hai loại tìm kiếm. Cụ thể:

1. **Vector Search (Milvus) - Chiếm 70% trọng số:**

```21:25:src/agent.py
        vectorstore = connect_to_milvus('http://localhost:19530', collection_name)
        milvus_retriever = vectorstore.as_retriever(
            search_type="similarity", 
            search_kwargs={"k": 4}
        )
```

- Sử dụng Milvus để tìm kiếm dựa trên vector embeddings
- Lấy 4 kết quả gần nhất (k=4)
- Dùng OpenAI embeddings hoặc Ollama để tạo vector

2. **BM25 Search - Chiếm 30% trọng số:**

```27:37:src/agent.py
        # Tạo BM25 retriever từ toàn bộ documents
        documents = [
            Document(page_content=doc.page_content, metadata=doc.metadata)
            for doc in vectorstore.similarity_search("", k=100)
        ]
        
        if not documents:
            raise ValueError(f"Không tìm thấy documents trong collection '{collection_name}'")
            
        bm25_retriever = BM25Retriever.from_documents(documents)
        bm25_retriever.k = 4
```

- Tìm kiếm dựa trên từ khóa (keyword-based)
- Cũng lấy 4 kết quả (k=4)
- Hoạt động tốt với việc tìm kiếm chính xác từ khóa

3. **Kết hợp hai phương pháp:**

```40:43:src/agent.py
        ensemble_retriever = EnsembleRetriever(
            retrievers=[milvus_retriever, bm25_retriever],
            weights=[0.7, 0.3]
        )
```

- Vector search: 70% trọng số
- BM25: 30% trọng số

4. **Quá trình xử lý văn bản:**

```170:174:src/seed_data.py
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=30,
        chunk_overlap=5,
        length_function=len
    )
```

- Chia văn bản thành các đoạn nhỏ 30 ký tự
- Có 5 ký tự chồng lấp giữa các đoạn
- Mỗi đoạn được gán metadata

5. **Xử lý lỗi:**

```46:55:src/agent.py
    except Exception as e:
        print(f"Lỗi khi khởi tạo retriever: {str(e)}")
        # Trả về retriever với document mặc định nếu có lỗi
        default_doc = [
            Document(
                page_content="Có lỗi xảy ra khi kết nối database. Vui lòng thử lại sau.",
                metadata={"source": "error"}
            )
        ]
        return BM25Retriever.from_documents(default_doc)
```

- Nếu có lỗi kết nối database
- Trả về BM25 retriever với thông báo lỗi mặc định

Ưu điểm của cách tiếp cận này:
- Kết hợp được ưu điểm của cả tìm kiếm ngữ nghĩa (vector) và từ khóa (BM25)
- Vector search giúp hiểu ngữ cảnh và ý nghĩa
- BM25 đảm bảo tìm được các từ khóa chính xác
- Có xử lý lỗi và fallback mechanism
- Metadata giúp theo dõi nguồn gốc và vị trí của thông tin

Đây là một cách tiếp cận khá toàn diện, kết hợp nhiều kỹ thuật để đảm bảo kết quả tìm kiếm chính xác và phù hợp với ngữ cảnh.